In [2]:
import os
import csv 
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from langchain_together import Together
from langchain_community.document_loaders import CSVLoader

load_dotenv()

True

---

## Document Loading

In [77]:
file_path = "./Data/concatenated_28657_products_list.csv"
csv.field_size_limit(10**6)
loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ",",
    },
    source_column="parent_asin"
)

In [78]:
data = loader.load()

In [79]:
len(data)

28657

---

## Text Splitting

In [70]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter  = RecursiveCharacterTextSplitter(
    chunk_size = 1100,chunk_overlap=150,add_start_index=True
)
all_splits = text_splitter.split_documents(data)

In [71]:
len(all_splits)

41037

---

## Embeddings

In [4]:
from langchain_together import TogetherEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
embeddings_model_name = "multi-qa-mpnet-base-dot-v1"

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name,)

/mnt/Files/Sorbonne_Data_Analytics_SDA/Cours/Gen_AI_Dafnis-Krasniqi/Gift_Recommendation_ChatBot/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [23]:
text = all_splits[0]
text.page_content

'parent_asin: B08RBDB6ZT\naverage_rating: 4.6\nrating_number: 301'

In [81]:
data[0]

Document(metadata={'source': 'B07KFGWC8S', 'row': 0}, page_content="parent_asin: B07KFGWC8S\naverage_rating: 4.5\nrating_number: 1020\nmain_category: All Beauty\ntitle: Amope PediMask Kit- 20 Minute Foot Mask to Rejuvenate and Soothe Your Feet with Blend of Moisturizers and Macadamia Oils for Baby Smooth Feet in Minutes (Pack of 3)\ndescription: Your feet go through so much every day and have never failed you, isn't it time you show them some love? Amope PediMask aims at making your feet look and feel great with every step you take.\nfeatures: FOOT MASK VALUE PACK: Amopé PediMask Value Pack is the perfect option to add to your foot care routine for healthy looking feet just after one use RESULTS IN 20 MINUTES: The sock masks in this pack rejuvenate and soothe heels in just 20 minutes GOODNESS OF NATURAL OILS: Enriched with the goodness of moisturizers and Macadamia Oil, Amopé provides long lasting hydration to your feet EASY TO USE: The foot mask is designed to stay in place resulting 

In [73]:
%%time
vector = embeddings.embed_query(data[0].page_content)

CPU times: user 64.3 ms, sys: 15.5 ms, total: 79.8 ms
Wall time: 32 ms


In [84]:
from langchain_chroma import Chroma
persist_directory = './chroma_vectorstore/'
vectorstore = Chroma.from_documents(documents=data, embedding=embeddings,persist_directory=persist_directory,)

: 

In [5]:
### Load the vectorstore
from langchain_chroma import Chroma
persist_directory = './chroma_vectorstore/'
vectorstore = Chroma(persist_directory=persist_directory,embedding_function=embeddings)

---

## Retrieval and Generation

In [6]:
query = "I am looking for a gift for my 30 year old mom. She loves makeup and has a light skin. she loves the friends show"

In [7]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":6})
retrieved_docs=retriever.invoke(query)
len(retrieved_docs)

6

---

## RAG

In [8]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(base_url="https://api.together.xyz/v1",api_key=os.getenv("TOGETHER_API_KEY"),model="meta-llama/Llama-Vision-Free")

In [9]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())

In [10]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    input_key="question",
    memory_key="history",
    #output_key="result",
    return_messages=True,
)

/tmp/ipykernel_259710/3813494411.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [11]:
question = "i want a gift for my mother. it is for her birthday. she loves makeup. she is in her 50s"

In [50]:
from langchain.prompts import PromptTemplate

# Build prompt

template = """You are an intelligent gift recommendation assistant designed to help users find the perfect gifts based on their preferences, interests, and occasions. Your goal is to provide personalized gift suggestions using information from a retailer's product catalog and generate engaging responses to assist the user.

1. User Preferences:
Start by gathering details about the recipient, such as age, gender, interests, hobbies, and any relevant occasions (e.g., birthdays, anniversaries, holidays). This will help you tailor your suggestions more effectively.

2. Product Retrieval:
Use the retailer's catalog to find products that match the user’s criteria. Only recommend products from the catalog. Avoid suggesting any items not listed in the retailer's database.
Recommend up to 3 products that best suit the recipient’s profile. If no suitable options are found, state: "We don't have any recommendations that fit the given criteria."
Ensure recommendations come from different categories for variety.
3. Personalized Suggestions:
Provide clear and concise descriptions of each recommended item, including:

Features (color, size, price, etc.).
Benefits, explaining why it makes a great gift.
Target audience and any trending attributes (e.g., popularity, user ratings).
Include a link to each product using the format: 'https://amazon.com/dp/(ASIN)'.
4. Formatting:
Present recommendations in a well-organized format:

Recommendation Title: Include the product name.
Description: Briefly describe the product and its appeal.
Features and details: List key features (e.g., color, size, price).
Product link: Provide the formatted link.
5. Interaction and Follow-up Questions:
Encourage further user interaction by asking follow-up questions, such as:

Preferences for specific brands, styles, or colors?
Any budget limitations?
Use responses to refine and improve the recommendations.
6. Feedback Loop:
Invite users to rate the suggestions or provide comments to enhance future recommendations.

7. Tone and Style:
Maintain a friendly, conversational tone throughout.

Use simple, easy-to-understand language.
Avoid jargon and keep the conversation engaging.
Remember to keep the interaction smooth and adjust based on user feedback. Your main goal is to help the user find a delightful gift and make their shopping experience enjoyable.
Question: {question}
Context: {context}
history: {history}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)


# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT,"verbose":True,"memory":memory})

In [51]:
result = qa_chain({"query": question})

/tmp/ipykernel_29639/4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an intelligent gift recommendation assistant designed to help users find the perfect gifts based on their preferences, interests, and occasions. Your goal is to provide personalized gift suggestions using information from a retailer's product catalog and generate engaging responses to assist the user.

1. User Preferences:
Start by gathering details about the recipient, such as age, gender, interests, hobbies, and any relevant occasions (e.g., birthdays, anniversaries, holidays). This will help you tailor your suggestions more effectively.

2. Product Retrieval:
Use the retailer's catalog to find products that match the user’s criteria. Only recommend products from the catalog. Avoid suggesting any items not listed in the retailer's database.
Recommend up to 3 products that best suit the recipient’s profile. If no suitable options are found, state: "We don't have any recomm

In [52]:
result

{'query': 'i want a gift for my mother. it is for her birthday. she loves makeup. she is in her 50s',
 'result': "Happy birthday to your mom! I'd be delighted to help you find a lovely gift for her special day.\n\nBased on your mom's love for makeup and her age in her 50s, I've curated three gift suggestions from the Amazon catalog:\n\n**Recommendation 1:**\n**Makeup Brush Set**\nDescription: A set of high-quality, soft-bristled makeup brushes perfect for any makeup enthusiast.\nFeatures:\n\t* 12-piece brush set\n\t* Soft, synthetic bristles for gentle application\n\t* Ergonomic handles for comfortable grip\n\t* Great for beginners and experienced makeup artists alike\nTarget Audience: Makeup lovers, beauty enthusiasts\nASIN: B07BQ9RKNP\nhttps://amazon.com/dp/B07BQ9RKNP\n\n**Recommendation 2:**\n**Personalized Makeup Bag**\nDescription: A customized makeup bag with your mom's name or initials, perfect for storing her favorite makeup essentials.\nFeatures:\n\t* High-quality, durable mat

In [53]:
result = qa_chain({"query": "What do you remember from our conversation?"})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an intelligent gift recommendation assistant designed to help users find the perfect gifts based on their preferences, interests, and occasions. Your goal is to provide personalized gift suggestions using information from a retailer's product catalog and generate engaging responses to assist the user.

1. User Preferences:
Start by gathering details about the recipient, such as age, gender, interests, hobbies, and any relevant occasions (e.g., birthdays, anniversaries, holidays). This will help you tailor your suggestions more effectively.

2. Product Retrieval:
Use the retailer's catalog to find products that match the user’s criteria. Only recommend products from the catalog. Avoid suggesting any items not listed in the retailer's database.
Recommend up to 3 products that best suit the recipient’s profile. If no suitable options are found, state: "We don't have any recomm

In [54]:
result

{'query': 'What do you remember from our conversation?',
 'result': "It seems like we had a previous conversation about finding a gift for your mother's birthday. She's in her 50s and loves makeup. I provided three gift suggestions:\n\n1. **Makeup Brush Set**: A 12-piece brush set with soft, synthetic bristles and ergonomic handles.\n2. **Personalized Makeup Bag**: A customized makeup bag with her name or initials, perfect for storing her favorite makeup essentials.\n3. **Makeup Storage Tray**: A stylish and functional tray for organizing and storing makeup products.\n\nHowever, it seems like we've started a new conversation. I'm not sure if you'd like to continue with the previous suggestions or start fresh. Could you tell me more about what you're looking for this time?",
 'source_documents': [Document(metadata={'row': 13168, 'source': './Data/concatenated_output.csv', 'start_index': 0}, page_content="parent_asin: B0C1FX3BMW\naverage_rating: 4.8\nrating_number: 5377\ntext: Toys & Gam

---

## Output Streaming